# Tutorial #13: FITS images

Let's repeat some of what we did during lecture with a new image. This time in X-ray

In [ ]:
data_url = 'https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/2025_01/00018938039/xrt/products/sw00018938039xpc_sk.img.gz'

Open the fits file at data_url, and inspect it. 

- How many HDUs are there?
- display the header info
- what telescope and/or instrument took this image?
- How many images are there?
- What is the size of the image(s)?
- What is the exposure time of the image(s) in seconds?

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np


Using astropy's WCS module, find the RA and Dec of the pixel with the largest value

Plot the image with sky coordinates as the axes

- Use a colormap that helps you vizualize the data best
- follow the guide [here](https://docs.astropy.org/en/latest/visualization/wcsaxes/ticks_labels_grid.html#tick-label-format) to change the format of the axis labels, make them in degrees with 2 decimal points ('d.dd')

Find all the pixels that are well above background 

- Get the indexes where pixels have more than 5 counts
- Get the ra and dec of those indexes
- Find the max and min values of those ra's and dec's

Read back in the fits file from today's lecture

In [ ]:
sdss_image_file = fits.open('https://cdsarc.cds.unistra.fr/saadavizier/download?oid=864972989978903253')
sdss_wcs = WCS(sdss_image_file[0].header)

Plot it again like we did in class with RA and Dec as the x and y axes. 

- Use the same label format as the previous part, 'd.dd'
- Use whatever colormap you like

Plot it one more time, but this time also plot where there are bright X-rays

- Use ax.scatter(ra_values, dec_values, transform=ax.get_transform('icrs')) to make a marker where the bright X-rays are

Where are the bright X-rays in relation to the object seen in the optical image?

NGC 4151 is a Seyfert galaxy, google what a Seyfert galaxy is and see if you can make sense of why the X-rays are where they are and why the optical photons are more spread out